### Analysis of the DataSet

In [4]:
import pandas as pd
import numpy as np
import networkx as nx

### Preprocessing

In [5]:
def getSize(fileName):
    dataFrame = pd.read_csv(fileName,delimiter=',')
    traderList = list(dataFrame['Seller'])
    traderList.extend(list(dataFrame['Buyer']))
    traderSet = set(traderList)
    traderSize = len(traderSet)
    return traderSize

### Undirected Graph generation

In [6]:
# total transaction amount between them is the weight
def getUndirectedGraph(fileName):
#     K represents k nearest neighbors
    dataFrame = pd.read_csv(fileName, delimiter =',')
    nodeSize = getSize(fileName)
    myGraph = nx.Graph()
#     Added nodes to the Graph
    myGraph.add_nodes_from(list(range(1, nodeSize+1)))
#    Adding edges with weights to the Graph
    for ind in dataFrame.index:
        u = dataFrame['Seller'][ind]
        v = dataFrame['Buyer'][ind]
        w = dataFrame['Amt'][ind]
        if u in list(myGraph.adj[v]):
            myGraph[u][v]['weight']+=w
        else :
            myGraph.add_edge(u, v, weight = w)
    return myGraph

undirectedGraph = getUndirectedGraph('dataset.csv')

### Directed Graph generation

In [10]:
def getDirectedGraph(fileName):
    dataFrame = pd.read_csv(fileName, delimiter =',')
    nodeSize = getSize(fileName)
    myGraph = nx.DiGraph()
#     Added nodes to the Graph
    myGraph.add_nodes_from(list(range(1, nodeSize+1)))
#    Adding edges with weights to the Graph
    for ind in dataFrame.index:
        u = dataFrame['Seller'][ind]
        v = dataFrame['Buyer'][ind]
        w = dataFrame['Amt'][ind]
        if u in list(myGraph.adj[v]):
            myGraph[v][u]['weight']+=w
        else :
            myGraph.add_edge(u, v, weight = w)
    return myGraph

DirectedGraph = getDirectedGraph('dataset.csv')

### Shared nearest neighbour Clustering Algorithm

### K-Nearest Neighbour

In [35]:
%%time
def kNear(graph, K):
    Dict = {}
    nodeSize = getSize('dataset.csv')
    for node in range(1,nodeSize+1):
        neighbors = np.array(sorted(graph[node].items(), key=lambda e: e[1]["weight"], reverse=True))
        if(neighbors.shape[0]<K):
            neighborList = [x[0] for x in neighbors]
        else : 
            neighbors = neighbors[:K]
            neighborList = [x[0] for x in neighbors]
        Dict[node]=neighborList
    return Dict

knnGraph = kNear(DirectedGraph, 4)

CPU times: user 989 ms, sys: 78.6 ms, total: 1.07 s
Wall time: 883 ms


### Union find algorithm

In [50]:
nodeSize = getSize('dataset.csv')
clusterId = np.array(list(range(nodeSize+1)))
clusterSize = np.ones(nodeSize, dtype = int)
def parent(u):
    while clusterId[u]!=u:
        u = clusterId[u]
    return u
def common(u, v):
    neighborU = knnGraph[u]
    neighborV = knnGraph[v]
    cnt = 0 
    for x in neighborU:
        if x in neighborV:
            cnt = cnt+1
    return cnt
def union(u, v):
    x = parent(u)
    y = parent(v)
    if clusterSize[x]>clusterSize[y]:
        clusterId[y] = x
        clusterSize[x]+=clusterSize[y]
    else :
        clusterId[x] = y 
        clusterSize[y]+=clusterSize[x]

### Shared Nearest Neighbor Algorithm

In [52]:
%%time
kt = 2
for u in range(1, nodeSize+1):
    for v in range(1, nodeSize+1):
        x = parent(u)
        y = parent(v)
        if x!= y :
            if u in knnGraph[u] and v in knnGraph[u]:
                if common(u,v)>=kt:
                    union(u,v)
# union find
# set - > all nodes, each node as it's own cluster
# for every (u,v) in G : 
# if v not in C(u) && if k(u) && k(v)>= kt && u->knn(V) && v->knn(u):
# remove c(u), c(v) 
# add c(u)union c(v) to set S

KeyboardInterrupt: 

### Final dictionary of Shared nearest neighbor

In [57]:
SNNClust = {}
vis = np.zeros(nodeSize+1, dtype = int)
for node in range(1, nodeSize+1):
    currId = parent(node)
    if vis[currId]==1:
         SNNClust[currId]=[node]
    else : 
        SNNClust[currId].append(node)
    vis[currId] = 1

KeyError: 1

{1: [2, 3]}


### Playing with NetworkX

In [ ]:
import networkx as nx

In [ ]:
# Creating a graph
G = nx.Graph()

In [ ]:
# Add nodes in the graph
G.add_node(1)
G.add_nodes_from([2,3])

In [ ]:
# Add edges in the graph with weight
G.add_edge(1, 2, weight=4.7 )
G.add_edge(1,3, weight = 2.3)

In [ ]:
# Accessing and modifying weight of a graph
G[1][2]['weight']+=2

In [ ]:
# Getting number of nodes and edges in the graph
print(G.number_of_nodes())
print(G.number_of_edges())

In [ ]:
# Node list and Edge List
# Neighbor List
nodeList = list(G.nodes)
edgeList = list(G.edges)
n1List = list(G.adj[1])
n2List = list(G.adj[2])
n3List = list(G.adj[4])
print(nodeList)
print(edgeList)
print(n1List)
print(n2List)
print(n3List)

In [ ]:
# Remove nodes and edges
G.remove_edge(1,2)

In [8]:
# Creation of diGraph
# edges will be doubled
H = nx.DiGraph()
H.add_node(4)
H.add_edge(1,2, weight=2.3)
H.add_edge(1,2, weight = 3.2)
H.add_edge(2, 1, weight=3.2)
H.add_edge(1, 3, weight = 4.5)
print(list(H.adj[4]))
for (u, v, wt) in H.edges.data('weight'):
    print('(%d, %d, %.3f)' % (u, v, wt))
print(list(H.edges))
S = H.to_undirected()
for (u, v, wt) in S.edges.data('weight'):
    print('(%d, %d, %.3f)' % (u, v, wt))
print(list(S.edges))

[]
(1, 2, 3.200)
(1, 3, 4.500)
(2, 1, 3.200)
[(1, 2), (1, 3), (2, 1)]
(1, 2, 3.200)
(1, 3, 4.500)
[(1, 2), (1, 3)]


In [30]:
# creationg of normal Graph
# edges weight will be added
S = nx.Graph()
S.add_edge(1,2, weight=2.3)
S.add_edge(1,4, weight = 2.4)
S.add_edge(2, 1, weight=3.2)
S.add_edge(1, 3, weight = 2.0)
S.add_edge(2,3,weight=2.8)
S[1][2]['weight']+=2.0
S[2][1]['weight']+=10.0
print(S[1][2]['weight'])
foo = np.array((sorted(S[1].items(), key=lambda e: e[1]["weight"], reverse=True)))[:K]
print(foo.shape)
print(foo)
for (x, y) in foo:
    print(x)

15.2
(3, 2)
[[2 {'weight': 15.2}]
 [4 {'weight': 2.4}]
 [3 {'weight': 2.0}]]
2
4
3


In [ ]:
# Accessing edges and neighbors
print(G[1][3]['weight'])
print(G[1])